In [1]:
import requests
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
ACCESS_TOKEN = "BQCL0Qpd86x6arlyx9NmlaD60q2fcm3QT8qGC_Q1-Ya1j9Os_0vC41fxxW9GcrxfA598sb1Ro103GHG3J1R7b7RcR0JQAkh7B1Pq-lphuKYby5Q7emg"

In [3]:
def search_song_on_spotify(song_name):
    headers = {
        'Authorization': f'Bearer {ACCESS_TOKEN}'
    }
    url = f'https://api.spotify.com/v1/search?q={song_name}&type=track&limit=50'
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        tracks = data['tracks']['items']
        return [(track['id'], track['name'], track['artists'][0]['name']) for track in tracks]
    else:
        print(f"Error fetching data: {response.status_code}")
        return []

In [4]:
def get_audio_features(track_ids):
    headers = {
        'Authorization': f'Bearer {ACCESS_TOKEN}'
    }
    ids = ','.join(track_ids)
    url = f'https://api.spotify.com/v1/audio-features?ids={ids}'
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        return data['audio_features']
    else:
        print(f"Error fetching audio features: {response.status_code}")
        return []

In [5]:
def analyze_bass_characteristics(track_name):
    results = search_song_on_spotify(track_name)
    
    if not results:
        print("No tracks found.")
        return
    
    track_ids = [track[0] for track in results]
    
    audio_features = get_audio_features(track_ids)
    
    audio_features = [features for features in audio_features if features is not None]
    
    if not audio_features:
        print("No audio features found.")
        return
    
    # Extract all features (except liveness cuz wth is that)
    feature_vectors = np.array([[features['danceability'], 
                                 features['energy'], 
                                 features['valence'], 
                                 features['loudness'], 
                                 features['key'], 
                                 features['mode'], 
                                 features['speechiness'], 
                                 features['acousticness'], 
                                 features['instrumentalness'], 
                                 features['tempo'], 
                                 features['time_signature']] 
                                for features in audio_features])
    
    # I want to give most importance to energy, danceability and valence
    # so scale them up
    prioritized_features = feature_vectors.copy()
    
    prioritized_features[:, 0] *= 2  # Danceability
    prioritized_features[:, 1] *= 2  # Energy
    prioritized_features[:, 2] *= 2  # Valence
    
    # Calculate similarities
    similarities = cosine_similarity(prioritized_features)
    
    # Find the indices of the most similar tracks (excluding itself)
    most_similar_indices = np.argsort(similarities[0][1:])[-10:][::-1] + 1
    
    slowed_sped_tracks = []
    different_songs_tracks = []
    
    # Categorize tracks based on their names
    for index in most_similar_indices:
        similar_track = results[index]
        track_name, artist_name = similar_track[1], similar_track[2]
        
        # Check if the track name contains keywords indicating slowed or sped versions
        if any(keyword in track_name.lower() for keyword in ['slowed', 'sped up', 'reverbed']):
            slowed_sped_tracks.append(f"{track_name} by {artist_name}")
        else:
            different_songs_tracks.append(f"{track_name} by {artist_name}")
    
    print(f"Top Similar Tracks to '{track_name}':")
    
    if slowed_sped_tracks:
        print("\nSlowed/Sped/Reverbed Versions:")
        for track in slowed_sped_tracks:
            print(f"- {track}")
    
    if different_songs_tracks:
        print("\nDifferent Similar Songs:")
        for track in different_songs_tracks:
            print(f"- {track}")

In [7]:
track_name_input = input("Enter a song name to analyze its bass characteristics on Spotify: ")
analyze_bass_characteristics(track_name_input)

Top Similar Tracks to 'MONTAGEM DIAMANTE ROSA':

Slowed/Sped/Reverbed Versions:
- MONTAGEM CORAL 1.0 - Super Slowed by SHADXWLXRD
- MONTAGEM CORAL 1.0 (STADIUM SLOWED) by xxxcharacter
- Montagem Mysterious Game - Slowed by LXNGVX
- MONTAGEM CORAL 1.0 (SLOWED) by xxxcharacter
- MONTAGEM CORAL 1.0 - Sped Up by SHADXWLXRD
- MONTAGEM DIAMANTE ROSA - SLOWED by vtze archive

Different Similar Songs:
- Montagem Mysterious Game by LXNGVX
- MONTAGEM CORAL 1.0 - HE'S BACKKK by SHADXWLXRD
- Montagem Coral 1.0 - Techno Down by F4ST
- MONTAGEM DIAMANTE ROSA by vtze archive
